In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from scipy.stats import multivariate_normal

def load_confusion_data_pickle(input_path="confusion_Kc.pkl"):
	with open(input_path, "rb") as f:
		data = pickle.load(f)
	print(f"Confusion data loaded from {input_path}")
	return data

# Load confusion matrix K
K = load_confusion_data_pickle("confusion_Kc=60.pkl")

# Confusion vector (diagonal of K)
kappa = np.diag(K)

Confusion data loaded from confusion_Kc=60.pkl


In [6]:
# Constants
epsilon = 0.0016  # Differential power value
sigma = 0.0046    # Standard deviation of noise
snr_factor = epsilon / (2 * sigma)  # Signal-to-noise ratio
snr_factor_squared = snr_factor**2  

mu = 0.5 * (epsilon / sigma) ** 2 * kappa
Sigma = (epsilon / sigma) ** 2 * K + 0.25 * (epsilon / sigma) ** 4 * (K - np.outer(kappa, kappa))

# Check the eigenvalues of Sigma to see if it's positive definite
eigvals = np.linalg.eigvals(Sigma)

# Output the eigenvalues
print(f"Eigenvalues of Sigma: {eigvals}")

# Check if all eigenvalues are positive
if np.all(eigvals > 0):
    print("Sigma is positive definite.")
else:
    print("Sigma is not positive definite.")

# Check if all eigenvalues are non-negative (positive semi-definite)
if np.all(eigvals >= 0):
    print("Sigma is positive semi-definite.")
else:
    print("Sigma is not positive semi-definite.")

Eigenvalues of Sigma: [ 2.09629704e+00  3.38238985e-01  2.38652657e-01  1.55987001e-01
  7.07783607e-02  7.03785511e-02  6.99338613e-02  6.94804601e-02
  5.54154421e-02  1.23685042e-02  3.02090812e-02  3.20731525e-02
  3.03957549e-02  3.18335455e-02  3.17837180e-02  3.05778166e-02
  3.06523461e-02  3.06949280e-02  3.08292233e-02  3.16648423e-02
  3.16027327e-02  3.09581512e-02  3.15054598e-02  3.14416531e-02
  3.13093344e-02  3.11341767e-02  3.11942829e-02  3.11793733e-02
  8.10827546e-03  7.49493720e-03  7.54817654e-03  8.02259428e-03
  7.58440124e-03  7.98609680e-03  7.95790303e-03  7.94654835e-03
  7.62533835e-03  7.63422403e-03  7.66959038e-03  7.68225111e-03
  7.69399368e-03  7.90435730e-03  7.72582483e-03  7.74003177e-03
  7.87712136e-03  7.86821831e-03  7.75971012e-03  7.83722384e-03
  7.83228160e-03  7.81138217e-03  7.78672924e-03  7.79905776e-03
  3.49754832e-17 -3.25037385e-17 -2.20252963e-17  2.00341372e-17
  1.77792272e-17  1.15063535e-17 -1.33480910e-17  4.17807291e-18
 -3

In [ ]:
rv = multivariate_normal(mean=mu, cov=Sigma)

In [ ]:
# Compute the matrix: K + (ε / 2σ)^2 * (K - κκ^T)
kappa_outer = np.outer(kappa, kappa)
adjusted_matrix = K + snr_factor_squared * (K - kappa_outer)

# sqrt
sqrt_matrix = sqrtm(adjusted_matrix)

# Inverse of the adjusted matrix
matrix_sqrt_inv = np.linalg.inv(sqrt_matrix)

term = snr_factor * matrix_sqrt_inv @ kappa

# Number of measurements
n_measurements = np.arange(1, 1001)

# Calculate SR_63
success_rate_sr63 = []
for n in n_measurements:
	x = np.sqrt(n) * term
	print(x)
	sr = rv.cdf(np.sqrt(n) * term)
	success_rate_sr63.append(sr)

# Plot SR_63
plt.figure(figsize=(10, 6))
plt.plot(n_measurements, success_rate_sr63, label="SR_63 (Theoretical)", color="blue")
plt.xlabel("Number of Measurements")
plt.ylabel("Success Rate")
plt.title("Success Rate SR_63 vs Number of Measurements")
plt.legend()
plt.grid()
plt.show()



LinAlgError: When `allow_singular is False`, the input matrix must be symmetric positive definite.